### Autor: Paula Domínguez Morales (855681)
### Fecha: 20 de Enero de 2025

---

<p style="text-align: center;">
    <img src="img/Uax_Logo.jpg" alt="Logo UAX" style="width: 40%;">
</p>

---

# Termofluidodinámica

## Estudio de una Caldera para Oxicombustión de Metano en Plantas de Cogeneración

---

### Introducción

Este notebook documenta la resolución de los requisitos establecidos para evaluar el diseño y el funcionamiento de una caldera basada en oxicombustión de metano. Este análisis abarca desde las condiciones iniciales de arranque hasta los modos estacionarios de operación. También se explorarán aspectos clave como la eficiencia térmica, la generación de contaminantes y los posibles ajustes en el diseño para optimizar su rendimiento.

El enunciado completo, con los detalles técnicos y los parámetros necesarios, se encuentra en el archivo adjunto dentro de la misma carpeta que este notebook. A continuación, se presenta un esquema de los puntos principales a resolver:

---

### Modelo de Arranque:
1. **Cálculo de la temperatura teórica del quemador** para diferentes dosados (φ) hasta alcanzar el régimen estequiométrico.
2. **Determinación de posibles cantidades de metano sin quemar** durante el arranque.
3. **Verificación de las condiciones necesarias para la ignición espontánea**, considerando alternativas para el arranque.
4. **Cálculo del área del cambiador de calor** para cumplir con los criterios de temperatura en el mezclador.
5. **Estimación del retraso máximo permitido** entre el inicio de la llama de premezcla y el cierre de la escotilla.

---

### Modos Estacionarios:
1. **Análisis de las configuraciones de funcionamiento**: 
   - *Modo A*: Condensador activado.
   - *Modo B*: Condensador desactivado.
2. **Evaluación de la cantidad de CH₄ y H₂O en recirculación** para cada modo.
3. **Cálculo de la cantidad de CO₂ recirculado** en cada configuración.
4. **Estudio de la longitud de la llama**, considerando si se desarrollan estructuras de Burke-Schumann.
5. **Impacto de los modos de funcionamiento en la operación**, incluyendo ventajas y desventajas entre las configuraciones A y B.

---

### Viabilidad y Limitaciones del Diseño:
Se discutirá la viabilidad de implementar este diseño de caldera para oxicombustión, evaluando aspectos como:

1. **La efectividad del condensador**.
2. **La idoneidad del metano como combustible**.
3. **La necesidad de redimensionar componentes clave**.
4. **Beneficios y limitaciones de los modos de operación propuestos**.

---

<p style="text-align: center;">
    <img src="img/Caldera.jpg" aldth: 70%;">
</div>

---


## Solución

#### Carga de librerías

In [ ]:
import math
from matplotlib import pyplot as plt
from pyturb.gas_models.semiperfect_ideal_gas import SemiperfectIdealGas

aire = SemiperfectIdealGas('Air')
Maire = aire.Mg*1e-3

#### Datos y propiedades de gas:

In [41]:
MC = 12e-3 # kg/mol
MN = 14e-3 # kg/mol
MO = 16e-3 # kg/mol
MH = 1e-3 # kg/mol

MCO2 = MC + 2*MO
MH2O = 2*MH + MO
MCH4 = MC + 4*MH
MO2 = 2*MO

#### Condiciones iniciales y de contorno

In [4]:
Tamb = 278.93 #K
pamb = 101325 #Pa
#V = 0.45 #m^3
#Tmed = 750 #K
Rg = 8.314 #J/K mol

#### Cinética química

In [5]:
A0 = 6e9 #(mol/cm3/s)**)(1-a-b)
a = -0.3
b = 1.3
Ea = 30 # kcal/mol
Ta = Ea*4184 / aire.Ru

#### Combustible: Metano

In [6]:
ratio_equiv = 0.25
LHV_metano = 50.0205e6 #J/kg
HF_metano = 891e3 #J/mol
alpha = 1
beta = 4
gamma = 0

#### Datos Caldera

In [44]:
Tmax = 2500 #K
Da = 50
Vcaldera = 3.6e-3 #m3
Lcaldera = 0.2 #m
Acaldera = Vcaldera / Lcaldera
P0 = 101325 #Pa
P0_atm = P0*9.869e-6
Pmax = 10e6 #Pa
T0 = 273+25 #K
Q_cc1 = -27500 #W Calor disipado en el cambiador de calor
Q_ch4_0 = 4500 #cc/s CAUDAL
Q_co2_0 = Q_ch4_0 #CAUDAL
Ecorriente = 100000 #J #Energía turbina
h_caldera = 1 #W/(m2 K) #Factor de conveccion caldera
h_cc = 35 #W/(m2 K) #Factor de conveccion cambiadores de calor

In [8]:
def estequiometria_hc(alpha,beta,gamma):
    """
    Calcula la estequiometría de una alcano asumiendo CnH2n+2Om donde n es el número de carbonos y m el de oxígenos presentes en el hidrocarburo.
    nFuel es la cantidad (mol) de combustible empleado. Si no se especifica se toma 1mol.
    """
    alfa = nCarb
    beta = nCarb*2+2
    gamma = nOxig
    
    nO2 = (alfa+beta/4-gamma/2)
    #nN2 = nO2 * 79/21 
    nCO2 = alfa
    nH2O = beta/2
    
    return nO2, nN2, nCO2, nH2O

In [9]:
def dosado_estequiometrico(alpha, beta, gamma):
    """
    Calcula el dosado estequiométrico de un HC.
    """

    nO2 = (alfa+beta/4-gamma/2)*nFuel
    #nN2 = nO2 * 79/21 
    nCO2 = alfa*nFuel
    nH2O = beta/2
    
    Mfuel = alfa*MC + beta*MH #kg/mol
    MComburente = nO2*(MO2)
    
    fs = Mfuel / MComburente
    
    return fs

## Cálculos iniciales:

La situación describe una oxicombustión de metano, por lo que el primer paso es determinar la mezcla estequiométrica de los gases involucrados. 

#### Parte estequiométrica de la reacción

In [ ]:
nO2_s, nN2_s, nCO2_s, nH2O_s = estequiometria_hc(1,4,0)

In [ ]:
print('Condición estequiométrica:')
print('nO2_s={:6.3f}mol;  nN2_s={:6.3f}mol;  nCO2_s={:6.3f}mol;  nH2O_s={:6.3f}mol'.format(nO2_s, nN2_s, nCO2_s, nH2O_s))

#### Condiciones de dosado estequiométrico

f = dosado_estequiometrico(1,4,0)

print('Dosado estequiométrico:', f)

## Arranque

In [13]:
#Teoría 1:

phi_pobre = np.linspace(0.1, 1, 10)
print ('phi mezcla pobre=', phi_pobre)

#para mezcla pobre
mcomb_pobre=mcomb_stq
Qneto = 0.6 * (mcomb_pobre * LHV_ch4)

No2_pobre_p = No2_stq/phi_pobre - alpha - beta/4

Tad_pobre = ( T0 + (Qneto)/(o2.cp_molar(T0)*No2_pobre_p+co2.cp_molar(T0)*Nco2_stq+h2o.cp_molar(T0)*Nh2o_stq))

print ('Barrido de temperaturas en mezcla pobre', Tad_pobre)

In [13]:
# teroía 2: mas aceptable

phi_pobre = np.linspace(0.1, 1, 10)
print ('phi mezcla pobre=', phi_pobre)

Nch4_pobre = phi_pobre*Nch4_stq
mcomb_pobre = Nch4_pobre * MCH4

Qneto = 0.6 * ((mcomb_pobre * LHV_ch4))
No2_stq_pobre = Nch4_pobre*(alpha+beta/4)
No2_pobre_p = No2_stq - No2_stq_pobre
Tad_pobre = ( T0 + (Qneto)/(o2.cp_molar(T0)*No2_pobre_p+co2.cp_molar(T0)*Nco2_stq+h2o.cp_molar(T0)*Nh2o_stq))
print (Tad_pobre)

In [ ]:
#para mezcla rica

phi_rica = np.linspace(1.1, 2, 10)
print ('phi mezcla rica=', phi_rica)

Nch4_rica = phi_rica*f_stq*ma_stq/mcomb_stq
Nch4_rica_p = Nch4_rica - Nch4_stq
Qneto = (0.6 * (mcomb_stq * LHV_ch4))
Tad_rica = T0 + Qneto/(ch4.cp_molar(T0)*Nch4_rica_p+co2.cp_molar(T0)*Nco2_stq+h2o.cp_molar(T0)*Nh2o_stq)
print( 'Barrido de temperaturas en mezcla rica ', Tad_rica)

In [ ]:
phi=np.concatenate((phi_pobre, phi_rica), axis=0)
Tad = np.concatenate((Tad_pobre, Tad_rica), axis=0)
print (phi)
print (Tad)
plt.plot(phi, Tad, color="green")
plt.title ( "t")
plt.xlabel("phi")
plt.ylabel("tad")

plt.annotate(str(Tad[15]), xy=(phi[15], Tad[15]))
plt.show() 

In [ ]:
# APArtado 2: calculo de los moles sobrantes de CH4

#FACCIONES MOLARES
Xch4_0 = 0.14470588
Xco2_0 = 0.2647059
Xh2o_0 = 0.5294117
Xo2_0 = 1 - (Xch4_0+Xco2_0+Xh2o_0)
phi_0 = 1.4
f_0 = phi_0*f_stq
print ('dosado (phi=1.4)', f_0)


N_total_stq = P0*Vc/(Rg*T0)

N_total_ch4_stq = N_total_stq / (1+(alpha + beta/4))

N_total_o2_stq = N_total_stq - N_total_ch4_stq


# esto es la mezcla estequiométrica para la cantidad REAL de moles en la caldera


# con estas formulas tenemos el sistema de eq entre suma total de moles, y relación de moles con el dosado 0.35
Nr_o2 = N_total_stq/(f_0*(MO2/(MCH4))+1)
Nr_ch4 = f_0 * (MO2/(MCH4))*Nr_o2

NrT = Nr_ch4 + Nr_o2

#Moles en el escape

NpT= NrT
Np_ch4 = NpT * Xch4_0
Np_co2 = Xco2_0*NpT
Np_h2o = Xh2o_0*NpT
Np_o2 = Xo2_0*NpT

NpT = Np_ch4+Np_co2+Np_h2o+Np_o2
print(f'Moles Totales en los reactantes = %-6.4f \nMoles Totales en los productos = %-6.4f'% (  NpT, NpT))

In [ ]:
print (f'Moles de CH4 reactantes = %-6.4f' %(Nr_ch4))
print (f'Moles de O2 reactantes = %-6.4f' %(Nr_o2))
print (f'Moles de CO2 productos = %-6.4f' %(Np_co2))
print (f'Moles de H2O productos = %-6.4f' %(Np_h2o))
print (f'Moles de CH4 productos = %-6.4f' %(Np_ch4)) 
print (f'Moles de O2 productos = %-6.4f' %(Np_o2))   

Xr_ch4 = Nr_ch4/NrT
print (f'Fracción molar CH4 reactante =%-6.4f'%Xr_ch4)
Xr_o2 = Nr_o2/NrT
print (f'Fracción molar O2 reactante =%-6.4f'%Xr_o2)

In [ ]:
# Cuenta de moles realmente quemados
Nq_o2 = Nr_o2 - Np_o2
print (f'Moles quemados de O2 = %-6.4f'%Nq_o2)
Nq_ch4 = Nr_ch4 - Np_ch4
Nq_r = Nq_ch4 + Nq_o2
print (f'Moles quemados de CH4 = %-6.4f'%Nq_ch4)
print (f'Moles totales quemados = %-6.4f'%Nq_r)


# Fracciones molares de lo quemado:
X_ch4_q = Nq_ch4/Nq_r
X_o2_q = Nq_o2/Nq_r
print (f'Fracción molar CH4 quemado = %-6.4f'%X_ch4_q)
print (f'Fracción molar de O2 quemado = %-6.4f'%X_o2_q)

In [ ]:
Moles quemados de O2 = 0.0776
Moles quemados de CH4 = 0.0393
Moles totales quemados = 0.1169
Fracción molar CH4 quemado = 0.3363
Fracción molar de O2 quemado = 0.6637

In [ ]:
# Comprobar que la temperatura y la presion del recinto del quemador
# son suficientes para que se genere una llama de difusion sin necesidad
# de pilotar la ignici´on. En caso de no ser suficiente, calcular la presi´on
# de la caldera inicial (presi´on de metano y ox´ıgeno) que deber´ıa haber
# para cumplir con la condici´on de encendido espont´aneo. Para estimar
# la condici´on de autoignici´on se puede asumir que la temperatura de la
# pared durante la operaci´on de la caldera es igual a la temperatura de
# pared calculada durante el arranque. ¿Se podr´ıa proporcional un ratio
# de quemado inicial diferente a ϕ = 1,4 y que permita igualmente iniciar
# la mezcla en modo llama de difusi´on?

In [ ]:
# apartado 3: primera estimación
EA = 30e3*4.184
Ta = EA / Rg
Qc_q = Nq_ch4 * Hc_ch4
Qneto_q = Qc_q * 0.6
# print ('Qc = ', Qc_q)

Qdisip_q = Qc_q * 0.4
# print ('Qdisip =', Qdisip_q)

Tpared_q =T0+(3.6e-4 * (Qdisip_q/(h * Ac)))
# print('Tpared =', Tpared_q)

Tc_q = (Ta - np.sqrt(Ta**2-4*Ta*Tpared_q))/2
# print ('Tc =', Tc_q)

# por comprobar que tenga sentido vamos a calcular la Tad
Tad_q = Tpared_q + Qneto_q/(ch4.cp_molar(Tpared_q)*Np_ch4+co2.cp_molar(Tpared_q)*Np_co2+h2o.cp_molar(Tpared_q)*Np_h2o+o2.cp_molar(Tpared_q)*Np_o2)

# print(Tad_q) #Muy alta pero concuerda con apartado 1

In [ ]:
a = -0.3
b = 1.3
A0 = 8.3e5

# Tc_q = 642
AA = Rg * Tc_q
BB = EA / ((a+b) * Rg * Tc_q)
CC = (h * Ac / Vc) * (Rg * Tc_q**2 / EA)
DD = Qneto_q * A0 * (X_ch4_q**a) * (X_o2_q**b)
EE = (1/(a+b))

Pc_q = AA * np.e**BB * (CC/DD)**EE

print ('Presion de comb',f' (Tc=%6.2f'%(Tc_q),f'K) = %-10.3ef'% Pc_q, 'Pa')
P_caldera = NrT*Rg*Tad_q/(Vc)
print (f'Presion de la caldera = %-10.3e'%P_caldera)
print ('Hay AutoComb? P_caldera>Pc_q : ', P_caldera>Pc_q )

In [ ]:
# Apartado 3, segunda espimación cambiando el dosado del apartado 2

#FACCIONES MOLARES
Xch4_0 = 0.14470588
Xco2_0 = 0.2647059
Xh2o_0 = 0.5294117
Xo2_0 = 1 - (Xch4_0+Xco2_0+Xh2o_0)
phi_0 = np.linspace(1.1, 2, 20)
f_0 = phi_0*f_stq
# 
# print ('phi = ', phi_0)


N_total_stq = P0*Vc/(Rg*T0)

N_total_ch4_stq = N_total_stq / (1+(alpha + beta/4))

N_total_o2_stq = N_total_stq - N_total_ch4_stq


# esto es la mezcla estequiométrica para la cantidad REAL de moles en la caldera


# con estas formulas tenemos el sistema de eq entre suma total de moles, y relación de moles con el dosado 0.35
Nr_o2 = N_total_stq/(f_0*(MO2/(MCH4))+1)
Nr_ch4 = f_0 * (MO2/(MCH4))*Nr_o2

NrT = Nr_ch4 + Nr_o2
Xr_o2 = Nr_o2/NrT
Xr_ch4 = Nr_ch4/NrT
#Moles en el escape

NpT= NrT
Np_ch4 = NpT * Xch4_0
Np_co2 = Xco2_0*NpT
Np_h2o = Xh2o_0*NpT
Np_o2 = Xo2_0*NpT

NpT = Np_ch4+Np_co2+Np_h2o+Np_o2
# print(f'Moles Totales en los reactantes = %-6.4f \nMoles Totales en los productos = %-6.4f'% (  NpT, NpT))

# Cuenta de mles realmente quemados
Nq_o2 = Nr_o2 - Np_o2
Nq_ch4 = Nr_ch4 - Np_ch4
Nq_r = Nq_ch4 + Nq_o2


# Fracciones molares de lo quemado:
X_ch4_q = Nq_ch4/Nq_r
X_o2_q = Nq_o2/Nq_r



EA = 30e3*4.184
Ta = EA / Rg
Qc_q = Nq_ch4 * Hc_ch4
rend = 0.6
Qneto_q = Qc_q * rend

Qdisip_q = Qc_q * (1-rend)

Tpared_q =T0+(3.6e-4 * (Qdisip_q/(h * Ac)))

Tc_q = (Ta - np.sqrt(Ta**2-4*Ta*Tpared_q))/2

# por comprobar que tenga sentido vamos a calcular la Tad
Tad_q = Tpared_q + Qneto_q/(ch4.cp_molar(Tpared_q)*Np_ch4+co2.cp_molar(Tpared_q)*Np_co2+h2o.cp_molar(Tpared_q)*Np_h2o+o2.cp_molar(Tpared_q)*Np_o2)

a = -0.3
b = 1.3
A0 = 8.3e5

# Tc_q = 642
AA = Rg * Tc_q
BB = EA / ((a+b) * Rg * Tc_q)
CC = (h * Ac / Vc) * (Rg * Tc_q**2 / EA)
DD = Qneto_q * A0 * (X_ch4_q**a) * (X_o2_q**b)
EE = (1/(a+b))

Pc_q = AA * np.e**BB * (CC/DD)**EE
P_caldera = NrT*Rg*Tad_q/(Vc)

phi_1 = phi_0[9]
Tad_1 = Tad_q[9]
T_pared_1 = Tpared_q[9]
Tc_1 = Tc_q[9]
Pc_1 = Pc_q[9]
P_caldera_1 = P_caldera[9]
Nq_ch4_1 = Nq_ch4[9]
Nq_o2_1 = Nq_o2[9]
NqT_1 = Nq_r[9] 
Nr_ch4_1 = Nr_ch4[9]
Nr_o2_1 = Nr_o2[9]
NrT_1 = NrT[9] 
Xr_ch4_1 = Xr_ch4[9]
Xr_o2_1 = Xr_o2[9]


print (f'phi minima =%2.3f'% phi_1)
print (f'Tad minima =%2.3f'%Tad_1 )
print (f'Tc minima =%2.3f'%Tc_1)
print (f'Pc minima =%1.3e'%Pc_1)
print (f'P_caldera minima =%1.3e'%P_caldera_1)



In [ ]:
# Apartado 4
# Calcular el ´area del cambiador de calor para que se cumpla el criterio
# de temperaturas en el mezclador

Acc = Qcc1/(hcc*(T_pared_1 - Tad_1))

print (f'Area del Cambiador de Calor = %1.3f m2'%(Acc))

In [ ]:
# Apartado 5
c0_ch4 = -9.1780e-3
c1_ch4 = 1.5964e-4
c2_ch4 = -3.4571e-8
c3_ch4 = 2.0839e-12
k_ch4 = c0_ch4 + c1_ch4*T0 + c2_ch4*T0**2 + c3_ch4*T0**3

c0_o2 = 5.9101e-4
c1_o2 = 1.0460e-4
c2_o2 = -3.7985e-8
c3_o2 = 7.372e-12
k_o2 = c0_o2 + c1_o2*T0 + c2_o2*T0**2 + c3_o2*T0**3

k_med = (k_ch4*Nr_ch4_1+k_o2*Nr_o2_1)/NrT_1
rho = P0/(T0*287)
cp_med =  ch4.cp_molar(T0)*(Xr_ch4_1/MCH4)+o2.cp_molar(T0)*(Xr_o2_1/MO2)
alpha_ = k_med/(cp_med*rho)

T_med = (Tad_1+T_pared_1)/2
rf = A0 * np.exp(-EA/(1600*Rg)) * Xr_ch4_1**a * Xr_o2_1**b * (P0*0.001/(1600*Rg))**(a+b)
tq = ((Xr_ch4_1*P0*0.001/(Rg*T0)))*0.001/rf

Sl = np.sqrt((alpha_*(Tad_1-Tc_1))/(tq*(Tc_1-T0)))

t = Lc/Sl

print (f'Velocida de la reacción =%2.4f'%rf)
print (f'Tiempo de reacción es =%2.4ef'%tq)
print (f'Velocidad del frente de llama =%2.4f'%Sl)
print (f'Tiempo para cerrar la valvula = %2.4f'%t)


#### Fracciones másicas y molares pedidas

In [15]:
frac_mol_Fuel = nFuel/nreact
frac_mol_O2 = nO2/nreact
frac_mol_N2 = nN2/nreact

mCO2 = nCO2*MCO2
mH2O = nH2O*MH2O
mN2 = nN2*(MN*2)
mO2prod = nO2_prod*(MO*2)

mprod = mCO2 + mH2O + mN2 + mO2prod

frac_mas_CO2 = mCO2 / mprod
frac_mas_H2O = mH2O / mprod
frac_mas_N2 = mN2 / mprod
frac_mas_O2prod = mO2prod / mprod

In [ ]:
print('Fracciones molares en reactantes:')
print('frac_molar_Fuel: {:6.4f};  frac_molar_O2: {:6.4f};  frac_masica_N2: {:6.4f}'.format(frac_mol_Fuel, frac_mol_O2, frac_mol_N2))
print('Suma fracciones: ', frac_mas_CO2+frac_mas_H2O+frac_mas_N2+frac_mas_O2prod)

In [ ]:
print('Fracciones másicas en productos:')
print('frac_masica_CO2: {:6.4f};  frac_masica_H2O: {:6.4f};  frac_masica_N2: {:6.4f};  frac_masica_O2: {:6.4f}'.format(frac_mas_CO2, frac_mas_H2O, frac_mas_N2, frac_mas_O2prod))
print('Suma fracciones: ', frac_mas_CO2+frac_mas_H2O+frac_mas_N2+frac_mas_O2prod)
frac_mas_CO2+frac_mas_H2O<10

#### Dosado real y estequiométrico

In [19]:
frac_mas_hept = (0.7*Mhept)/(0.7*Mhept+0.3*Mpent)
frac_mas_pent = (0.3*Mpent)/(0.7*Mhept+0.3*Mpent)

In [ ]:
fs = dosado_estequiometrico(7)*frac_mas_hept + dosado_estequiometrico(5)*frac_mas_pent
f = ratio_equiv*fs
print('f=',f)

#### Masa de productos asumida a aire

In [21]:
maire = mprod
naire = nprod

## Apartado 2:

Asumiendo que se tiene $1mol$ de combustible, calcule el calor de la combustión y la temperatura de llama. Para ello tome como temperatura promedio de la combustión 750K. Discuta la idoneidad de esta temperatura promedio.

#### Poder calorífico de combustible

In [22]:
LHVfuel = LHV_heptano*frac_mas_hept + LHV_pentano*frac_mas_pent

In [ ]:
print('LHVfuel = {:6.3f}MJ/kg'.format(LHVfuel*1e-6))

#### Temperatura adiabática de llama iterada

In [ ]:
sigue_iterando = True
Tmed = Tamb
iter_max = 100
tolerancia = 1e-6
niter = 0

while sigue_iterando:
    niter +=1
    Tmed_ = Tmed
    
    deltaTad = f*LHVfuel / ( (1+f)*aire.cp(Tmed) )
    
    Tad = Tamb + deltaTad
    Tmed = (Tad + Tamb)/2
    
    residuo = abs(Tmed-Tmed_)/Tmed_
    
    print('ii={};  Tmed={};  Tad={};  residuo={}'.format(niter, Tmed, Tad, residuo))
    
    if residuo<tolerancia:
        print('Solución obtenida')
        sigue_iterando = False
    elif niter>iter_max:
        print('Solución no convergida, iteraciones máximas excedidas')
        sigue_iterando = False


In [25]:
Tmed_ = 750 #K
deltaTad_ = f*LHVfuel / ( (1+f)*aire.cp(Tmed_) )
Tad_ = Tamb + deltaTad_
Tmed_ = 0.5*(Tad_+Tamb)

In [ ]:
print('Tad_={:7.1f}K'.format(Tad_))
print('Tmed_={:7.1f}K'.format(Tmed_))

La temperatura media propuesta es suficiente para caracterizar la mezcla de productos.

## Apartado 3

Calcule el tiempo característico para quemar $1mol$ de mezcla de combustible dadas las condiciones de presión y temperatura del combustor.

In [27]:
concentracionReact = (pamb/aire.Ru/Tamb) # mol/m^3
pre_exp = (A0*frac_mol_Fuel**a * frac_mol_O2**b * concentracionReact**(a+b))
exp = math.exp(-Ta/Tamb)

In [28]:
rf = pre_exp * exp
concentracionFuel = (pamb/aire.Ru/Tamb * frac_mol_Fuel)*1e-6
tau_q = concentracionFuel/rf

In [ ]:
print('rf={0:10.8f}mol/cm3/s'.format(rf))
print('tau_q={0:6.4}ms'.format(tau_q*1e3))

In [ ]:
concentracionFuel

## Apartado 4

Dado el resultado anterior discuta la idoneidad de la llama de premezcla para una turbina de gas. En caso de considerarla válida explique por qué, en caso contrario estudie el radio mínimo de inyección para tener una llama de difusión.

In [ ]:
nreact/(pamb/aire.Ru/Tamb)

In [ ]:
D0 = 2.475e3 # m^2/s
D = D0*math.exp(-Ta/Tmed)
print(D)

In [ ]:
math.sqrt(tau_q*D)